https://frhyme.github.io/machin-learning/sequence_classficiation/

In [11]:
import numpy as np
from keras.datasets import imdb

from keras.models import Sequential
from keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [2]:
np.random.seed(7) # fix random seed for reproducibility

In [5]:
"""
개별 movie review에 있는, 모든 단어를 고려하는 것은 무의미하기 때문에 
top_words, 즉 상위 5000개의 단어에 대해서만, 추려냄. 나머지는 필터링
그리고 단어는 index로 표시됨.
"""
# 리뷰 길이는 100, 참조할 단어는 빈도 상위 500개
max_review_length, top_words = 100, 500

np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = top_words)
np.load = np_load_old
"""
sequence의 길이를 똑같이 맞춤. 모두 0으로 채워넣음 
길이가 500보다 큰 경우에는 그냥 일괄적으로 앞부분을 잘라내버림. 
"""

X_train = sequence.pad_sequences(X_train, maxlen = max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen = max_review_length)

print("reading data complete")

reading data complete


In [15]:
"""
layer의 구성은 
- Embedding: 단어를 벡터화하고, 이 결과 값을 LSTM에 집어넣어줌. 
    - input_dim에 top_words를 넣어주는데, 아마도 내부에서 자동으로 one-hot vector를 만들어주는 것 같음
    - 현재는 one-hot vector가 아니라, 0, 1, 등 word vocab의 index가 넘어감. 
- Conv1D: 구조적인 특성을 파악하기 위해 여러 filter로 찍어줌.
- MaxPooling1D: convolution으로 찍어낸 정보를 좀 더 특징화함. 
- LSTM: sequential한 정보를 활용
- Dense: classification이므로 output layer을 1칸짜리로 넣어줌. 
"""
embedding_vector_length =32
model = Sequential([
    Embedding(input_dim = top_words, # 500
            output_dim = embedding_vector_length, # 32
            input_length = max_review_length),
    Conv1D(filters = 32, kernel_size = 5,padding='same', activation='relu'),
    MaxPooling1D(pool_size = 2),
    LSTM(50),
    Dropout(0.2),
    # 소프트맥스로 하는 것보다는, 0~1 중간값들을 살리기 위해 sigmoid
    Dense(25, activation='sigmoid'),
    Dense(1, activation = 'sigmoid')
])
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', 
              metrics =['accuracy'])
print(model.summary)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=64)
print("training complete")
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: {:.2f}".format(scores[1]*100))

<bound method Network.summary of <keras.engine.sequential.Sequential object at 0x000002273688C278>>
Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 56s 2ms/step - loss: 0.5418 - acc: 0.7142 - val_loss: 0.4446 - val_acc: 0.7968
Epoch 2/5
25000/25000 [==============================] - 56s 2ms/step - loss: 0.4223 - acc: 0.8064 - val_loss: 0.4128 - val_acc: 0.8124
Epoch 3/5
25000/25000 [==============================] - 56s 2ms/step - loss: 0.3989 - acc: 0.8204 - val_loss: 0.4001 - val_acc: 0.8195
Epoch 4/5
25000/25000 [==============================] - 57s 2ms/step - loss: 0.3837 - acc: 0.8300 - val_loss: 0.3931 - val_acc: 0.8206
Epoch 5/5
25000/25000 [==============================] - 58s 2ms/step - loss: 0.3680 - acc: 0.8360 - val_loss: 0.3869 - val_acc: 0.8224
training complete
Accuracy: 82.24


sig 한번만

Accuracy: 82.35

Accuracy: 82.24
